In [1]:
# 랭체인 사용하기
!pip install langchain chromadb langchain-openai pandas

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-4o",
    temperature = 0,
)

In [3]:
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# 데이터 불러오기
file_path = "d:/data/Disease_symptom_and_patient_profile_dataset.csv"
df = pd.read_csv(file_path)

# 데이터를 텍스트로 변환
df["text"] = df.apply(
    lambda row: f"질병: {row['Disease']}, 증상: 발열({row['Fever']}), 기침({row['Cough']}), 피로({row['Fatigue']}), 호흡 곤란({row['Difficulty Breathing']}), "
                f"연령: {row['Age']}, 성별: {row['Gender']}, 혈압: {row['Blood Pressure']}, 콜레스테롤: {row['Cholesterol Level']}, 결과: {row['Outcome Variable']}",
    axis=1
)

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings()

# hromaDB에 데이터 저장
chroma_path = "d:/data/chroma_disease_db"
vectorstore = Chroma.from_texts(df["text"].tolist(), embeddings, persist_directory=chroma_path)

# ChromaDB에서 검색할 리트리버 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# LangChain QA 체인 설정
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type="stuff")

# 사용자 질문 
query = "기침과 발열이 있는 질병은 무엇인가요?"
response = qa_chain.invoke(query)

# 결과 출력
print("📌 질문:", query)
print("💡 답변:", response)

📌 질문: 기침과 발열이 있는 질병은 무엇인가요?
💡 답변: {'query': '기침과 발열이 있는 질병은 무엇인가요?', 'result': '주어진 문맥에서는 발열은 있지만 기침은 없는 건선(Psoriasis)에 대한 정보만 제공되고 있습니다. 따라서 기침과 발열이 모두 있는 질병에 대한 정보는 제공되지 않았습니다. 일반적으로 기침과 발열은 감기, 독감, 코로나19 등의 호흡기 질환에서 흔히 나타나는 증상입니다. 하지만 정확한 진단을 위해서는 의료 전문가의 상담이 필요합니다.'}


In [5]:
query = "35대 여성이고, 기존에 혈압이 높았어, 어제부터 열이 있고 기침 증상이 있어. 무슨 질병을 의심해야해? 단, 데이터만 참조해서 답변해줘"
response = qa_chain.invoke(query)

# ✅ 8️⃣ 결과 출력
print("📌 질문:", query)
print("💡 답변:", response)

📌 질문: 35대 여성이고, 기존에 혈압이 높았어, 어제부터 열이 있고 기침 증상이 있어. 무슨 질병을 의심해야해? 단, 데이터만 참조해서 답변해줘
💡 답변: {'query': '35대 여성이고, 기존에 혈압이 높았어, 어제부터 열이 있고 기침 증상이 있어. 무슨 질병을 의심해야해? 단, 데이터만 참조해서 답변해줘', 'result': '제공된 데이터에 따르면, 35세 여성으로 고혈압이 있는 경우 발열과 기침 증상이 있을 때 결과는 "Negative"로 나타났습니다. 따라서, 이 데이터만을 참조했을 때 특정 질병을 의심할 수 있는 정보는 없습니다. 추가적인 의학적 평가가 필요할 수 있습니다.'}


In [7]:
!pip install llama-index langchain-openai pandas faiss-cpu

  Using cached llama_index-0.12.16-py3-none-any.whl.metadata (12 kB)
Using cached llama_index-0.12.16-py3-none-any.whl (6.9 kB)


In [7]:
#llamaindex 사용 방법
import pandas as pd
from llama_index.core import VectorStoreIndex, Document
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.settings import Settings

# CSV 데이터 불러오기
file_path = "d:/data/Disease_symptom_and_patient_profile_dataset.csv" # CSV 파일 경로 지정
df = pd.read_csv(file_path)  # CSV 파일을 Pandas DataFrame으로 로드

# 각 행(Row)에 대해 정보를 하나의 문자열로 합쳐서 저장
df["text"] = df.apply(
    lambda row: f"질병: {row['Disease']}, 증상: 발열({row['Fever']}), 기침({row['Cough']}), 피로({row['Fatigue']}), "
                f"호흡 곤란({row['Difficulty Breathing']}), 연령: {row['Age']}, 성별: {row['Gender']}, "
                f"혈압: {row['Blood Pressure']}, 콜레스테롤: {row['Cholesterol Level']}, 결과: {row['Outcome Variable']}",
    axis=1 # 각 행(Row)에 대해 적용
)

# OpenAI LLM 및 임베딩 모델 설정
Settings.llm = OpenAI(model="gpt-4o", temperature=0)
Settings.embed_model = OpenAIEmbedding()

# LlamaIndex 벡터 저장소에 데이터 추가
documents = [Document(text=text) for text in df["text"].tolist()] # CSV 데이터를 Document 객체로 변환
index = VectorStoreIndex.from_documents(documents) # 문서를 벡터화하여 저장소에 저장

# 검색 기능을 수행하는 query_engine 생성
query_engine = index.as_query_engine() # 질의응답 기능을 제공하는 엔진 생성

# 사용자가 입력한 질문을 기반으로 질의응답 수행
query = "기침과 발열이 있는 질병은 무엇인가요?"
response = query_engine.query(query) # LLM을 이용하여 질의응답 수행

# 결과 출력
print("📌 질문:", query)
print("💡 답변:", response)


📌 질문: 기침과 발열이 있는 질병은 무엇인가요?
💡 답변: 기침과 발열이 있는 질병은 제공된 정보에 없습니다.


In [8]:
query = "49대 여성이고, 기존에 혈압이 높고 높았어, 어제부터 열이 있고 기침 증상이 있어. 무슨 질병을 의심해야해? 단, 데이터만 참조해서 답변해줘"
response = query_engine.query(query)

# ✅ 8️⃣ 결과 출력
print("📌 질문:", query)
print("💡 답변:", response)

📌 질문: 49대 여성이고, 기존에 혈압이 높고 높았어, 어제부터 열이 있고 기침 증상이 있어. 무슨 질병을 의심해야해? 단, 데이터만 참조해서 답변해줘
💡 답변: Hypertension을 의심할 수 있습니다.
